In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imsave
import cv2
import json

In [ ]:
def crop_center(im:np.array, crop_width: int=20, crop_height: int=20) \
	-> np.array:
	"""
	im: 2D np array, single slice, 3 channels
	"""
	h, w, c = im.shape
	im = im[crop_height : h - crop_height, crop_width : w - crop_width, :]
	return im

In [4]:
def visualize_data(volume, annotations1, annotations2, slices, output_path, patient_id):
	colors = {0: (1, 0, 0), 1: (1, 0, 0), 2: (0, 1, 0), 3: (0, 0, 1), 4: (1, 1, 0)}
	
	imlist = []
	for i in slices:
		img1 = volume[i]
		img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
		annotations1_slice = [item for item in annotations1 if item[0]==i]
		print(f"auto annotations for slice {i}: {len(annotations1_slice)}")
		if len(annotations1_slice)>0:
			for class_idx in [1, 2, 3, 4]:
				contours = [np.array(item[2]).astype(np.int32) for item in annotations1_slice if item[1]==class_idx]
				print(f"contours for class {class_idx}: {len(contours)}")
				if len(contours)>0:
					img1 = cv2.drawContours(img1, contours, -1, colors[class_idx], 2, cv2.LINE_8)
					# img1 = cv2.polylines(img1, contours, 1, colors[class_idx], 1)

		img2 = volume[i]
		img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)
		annotations2_slice = [item for item in annotations2 if item[0]==i]
		print(f"manual annotations for slice {i}: {len(annotations2_slice)}")
		if len(annotations2_slice)>0:
			for class_idx in [1,2,3,4]:
				contours = [np.array(item[2]).astype(np.int32) for item in annotations2_slice if item[1]==class_idx]
				print(f"contours for class {class_idx}: {len(contours)}")
				if len(contours)>0:
					img2 = cv2.drawContours(img2, contours, -1, colors[class_idx], 2, cv2.LINE_8)

		full_im = np.concatenate((img1, img2), axis=0)
		imsave(os.path.join(output_path, f"{patient_id}_{i}.png"), (full_im * 255).astype(np.uint8))

In [67]:
root_path = "../outputs/autocontours_review_processed"
output_path = "../outputs/autocontours_review_results"

dicom_patient_ids = [1, 3, 6, 8]
# dicom_patient_ids = [3]
graded_slices = {1: np.arange(60, 124, 12).astype(np.int32),
				3: np.arange(63, 123, 12).astype(np.int32),
				6: np.arange(42, 106, 12).astype(np.int32),
				8: np.arange(35, 95, 12).astype(np.int32)}
for patient_id in dicom_patient_ids:
	print("Patient no: ", patient_id)
	patient_path = os.path.join(root_path, "{0:03d}_CT_EBRT".format(patient_id))
	image = np.load(os.path.join(patient_path, "image.npz"))["volume"]
	seg_auto = json.load(open(os.path.join(patient_path, "auto.json"), "r"))
	seg_manual = json.load(open(os.path.join(patient_path, "manual.json"), "r"))
	visualize_data(image, seg_auto, seg_manual, graded_slices[patient_id], output_path, patient_id)

Patient no:  1
auto annotations for slice 60: 6
contours for class 1: 1


error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/drawing.cpp:2428: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'polylines'


## Concat images

In [8]:
output_path = "../outputs/autocontours_review_results"
image_info = [(8, 35),
              (3, 63),
              (1, 60),
              (6, 54),
              (6, 94),
             ]
imlist = []
for info in image_info:
    patient_id, slice_idx = info
    filepath = os.path.join(output_path, f"{patient_id}_{slice_idx}.png")
    im = cv2.imread(filepath)
    imlist.append(im)

im = np.concatenate(imlist, axis=1)
cv2.imwrite(os.path.join(output_path, "oar_examples_new.jpg"), im)

True